<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/Fixed_BNN_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade -q gspread
from tensorboardcolab import *
import shutil
#clean out the directory
shutil.rmtree('./Graph', ignore_errors=True)
os.mkdir('./Graph')
tf.reset_default_graph()
#will start the tunneling and will print out a link:
tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://207f5966.ngrok.io


In [0]:
from google.colab import auth
auth.authenticate_user()
import gc

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [0]:
!git clone https://github.com/csamuelsson/bayesianNN
!mv bayesianNN/* .
!rm -rf bayesianNN

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function



In [0]:
# Dependencies
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import math
from sklearn.decomposition import PCA
from flags import *
#from absl import flags
import python_utils
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials


In [0]:
from IPython.display import display
import pandas as pd
from pathlib import Path

In [0]:
def read_worksheet(filename, sheetname):
    #read from Google Sheets
    rows = gc.open(filename).worksheet(sheetname).get_all_values()
    return rows

def read_worksheets(filename, sheetname, col_names):
    datasets = []
    for sheet in sheetname:
        rows = read_worksheet(filename,sheet)
        df = pd.DataFrame.from_records(rows,columns=col_names )
        datasets.append( df )
    return datasets
        

def merge_datasets(datasets):
    base = datasets[0]
    for ds in datasets[1:]:
        #print(len(ds))
        base = base.append(ds,ignore_index=True)
    return base

#def splitting_data(dataset, ratio_1 = 0.8, ratio_2 = 0.9, shuffle= True, seed = 0):
    #split_1 = int(ratio_1*len(datasets))
   # split_2 = int(ratio_2*len(datasets))
    #if shuffle:
        #np.random.seed(seed = seed)
       # np.random.shuffle(datasets.values)
    #train = datasets[:split_1]
    #validation= datasets[split_1:split_2]
    #test = datasets[split_2:]
    #return train, validation, test

#def split_dataset(dataset,ratio=0.3,shuffle=True,seed=0):
    #cnt = len(dataset)
    #cut = math.ceil(ratio*cnt)
    
    #if shuffle:
       # np.random.seed(seed=seed)
       # np.random.shuffle(dataset.values)
        
   # valid = dataset[:cut]
    #train = dataset[cut:]
    #return train,valid

def normalize(df,column):
    values = df[column].astype(float).values
    mean = np.mean(values)
    s = np.sum(values)
    std = np.std(values)
    norm_values = ( values - mean ) / std
    df[column] = norm_values
    return mean,std



def normalize_columns(df,col_names):
    means_stds = {}
    for col in col_names:
        means_stds[col] = normalize(df,col)
    return means_stds

In [0]:
#filename = 'data1.csv'
filename = 'dataset-baysian'
columns = 'columns'
sheets = ['data2','data23','data59','data6']


local_file = Path(filename+".csv")

unified = None

if not local_file.exists():
    col_names = read_worksheet(filename,columns)[0]
    datasets = read_worksheets(filename,sheets,col_names)
    #display( *[df.head() for df in datasets] )
    unified = merge_datasets(datasets)
    unified.to_csv(local_file)
else:
    unified = pd.read_csv(local_file)
labels = unified['class']
features = unified.drop(['class'], axis=1, inplace=False)
features 
labels 

In [0]:

# "class" != 1 => 0
unified['class'].replace(to_replace=r'[^1]+', value='0', inplace=True, regex=True)
labels = unified['class']
#norm_cols = ['time', 'messageID', 'pos/0', 'pos/1', 'pos/2', 'spd/0', 'spd/1']
#means_stds = normalize_columns(unified,norm_cols)

In [0]:
np.savez("data_main.npz", features=unified.values.astype(np.float32), labels=labels.values.astype(np.float32))

unified.head()

In [0]:
tfd = tf.contrib.distributions

"""Tuning program settings

FLAGS = flags.FLAGS
FLAGS.learning_rate = 0.09 # change
FLAGS.num_hidden_layers = 7
FLAGS.num_neurons_per_layer = 3
FLAGS.activation_function = "sigmoid"
FLAGS.num_principal_components = 7
FLAGS.batch_size = 1000     # kept constant under hyperopt
FLAGS.num_epochs = 3  # kept constant under hyperopt
TRAIN_PERCENTAGE = 0.8


"""
USE_PCA = False
FLAGS = flags.FLAGS
TRAIN_PERCENTAGE = 0.8

In [0]:
def build_input_pipeline(batch_size, number_of_principal_components):
    """Build an Iterator switching between train and heldout data.
    Args:
    `data_main.npz`: string representing the path to the .npz dataset.
    `batch_size`: integer specifying the batch_size for the dataset.
    `number_of_principal_components`: integer specifying how many principal components
    to reduce the dataset into.
    """
    # Build an iterator over training batches.
    with np.load('data_main.npz') as DATASET_FILE:
        features = DATASET_FILE["features"]
        labels = DATASET_FILE["labels"]
        features = features.astype(np.float32)
        labels = labels.astype(np.float32)
        
        # PCA (sklearn)
        if USE_PCA:
            features = PCA(n_components=number_of_principal_components).fit_transform(features)
        
        # Splitting into training and validation sets
        #train_range = int(TRAIN_PERCENTAGE * len(features))
        
        split_1 = int(TRAIN_PERCENTAGE*len(features))
        split_2 = int(TRAIN_PERCENTAGE+0.1 *len(features))
        
        training_features = features[:split_1]
        training_labels = labels[: split_1]
        validation_features = features[ split_1: split_2]
        validation_labels = labels[split_1:split_2]
        testing_features = features[split_2:]
        testing_labels      = labels[split_2:]
        
        # Z-normalising: (note with respect to training data)
        training_features = (training_features - np.mean(training_features, axis=0))/np.std(training_features, axis=0)
        validation_features = (validation_features - np.mean(training_features, axis=0))/np.std(training_features, axis=0)
        testing_features    = (testing_features -np.mean(training_features, axis=0))/np.std(training_features, axis=0)
    # Create the tf.Dataset object
    training_dataset = tf.data.Dataset.from_tensor_slices((training_features, training_labels))
    
    # Shuffle the dataset (note shuffle argument much larger than training size)
    # and form batches of size `batch_size`
    training_batches = training_dataset.shuffle(20000).repeat().batch(batch_size)
    training_iterator = training_batches.make_one_shot_iterator()
    
    # Build a iterator over the heldout set with batch_size=heldout_size,
    # i.e., return the entire heldout set as a constant.
    heldout_dataset = tf.data.Dataset.from_tensor_slices((validation_features, validation_labels))
    heldout_frozen = (heldout_dataset.take(len(validation_features)).
                    repeat().batch(len(validation_features)))
    heldout_iterator = heldout_frozen.make_one_shot_iterator()
    
    
    #Building iterator over the testing set 
    testing_dataset = tf.data.Dataset.from_tensor_slices((testing_features, testing_labels))
    testing_batches = testing_dataset.shuffle(20000).repeat().batch(batch_size)
    testing_iterator = training_batches.make_one_shot_iterator()
    
    
    # Combine these into a feedable iterator that can switch between training
    # validation,and testing inputs.
    # Here should the minibatch increment be defined 
    handle = tf.placeholder(tf.string, shape=[],)
    feedable_iterator = tf.data.Iterator.from_string_handle(
    handle, training_batches.output_types, training_batches.output_shapes)
    features_final, labels_final = feedable_iterator.get_next()
    
    return features_final, labels_final, handle, training_iterator, heldout_iterator, testing_iterator, split_1, split_2



In [0]:
def main(argv):
    # extract the activation function from the hyperopt spec as an attribute from the tf.nn module
    activation = getattr(tf.nn, FLAGS.activation_function)
    # define the graph
    with tf.Graph().as_default():
        (features, labels, handle,
         training_iterator, heldout_iterator, testing_iterator, train_range, valid_range) = build_input_pipeline(
            FLAGS.batch_size, FLAGS.num_principal_components)
        # Building the Bayesian Neural Network. 
        # We are here using the Gaussian Reparametrization Trick
        # to compute the stochastic gradients as described in the paper
        with tf.name_scope("bayesian_neural_net", values=[features]):
            neural_net = tf.keras.Sequential()
            for i in range(FLAGS.num_hidden_layers):
                layer = tfp.layers.DenseReparameterization(
                    units=FLAGS.num_neurons_per_layer,
                    activation=activation,
                    trainable=True,
                    kernel_prior_fn=tfp.layers.default_multivariate_normal_fn,
                    # NormalDiag with hyperopt sigmakernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
                    kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),  # softplus(sigma)
                    kernel_posterior_tensor_fn=lambda x: x.sample(),
                    bias_prior_fn=tfp.layers.default_multivariate_normal_fn,  # NormalDiag with hyperopt sigma
                    bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(),  # softplus(sigma)
                    bias_posterior_tensor_fn=lambda x: x.sample()
                )
                neural_net.add(layer)
            neural_net.add(tfp.layers.DenseReparameterization(
                units=1,  # one dimensional output
                activation=None,  # since regression (outcome not bounded)
                trainable=True,  # i.e subject to optimization
                kernel_prior_fn=tfp.layers.default_multivariate_normal_fn,  # NormalDiag
                kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),  # softplus(sigma)
                kernel_posterior_tensor_fn=lambda x: x.sample(),
                bias_prior_fn=tfp.layers.default_multivariate_normal_fn,  # NormalDiag
                bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(),  # softplus(sigma)
                bias_posterior_tensor_fn=lambda x: x.sample()
            ))
            
        predictions = neural_net(features)
        preds = []
        for _ in range(1000):
            preds.append(neural_net(features))
            MAP, var = tf.nn.moments(tf.squeeze(preds), axes=[0])
            
        # Compute the -ELBO as the loss, averaged over the batch size.
        neg_log_likelihood = tf.reduce_mean(tf.squared_difference(labels, predictions))
        kl = sum(neural_net.losses) / FLAGS.batch_size
        elbo_loss = kl + neg_log_likelihood
        # Build metrics for evaluation. Predictions are formed from a single forward
        # pass of the probabilistic layers. They are cheap but noisy predictions.
        accuracy, accuracy_update_op = tf.nn.softmax_cross_entropy_with_logits_v2(labels,predictions)

        with tf.name_scope("train"):
            # define optimizer - we are using (stochastic) gradient descent
            opt = tf.train.GradientDescentOptimizer(learning_rate=FLAGS.learning_rate)

            # define that we want to minimize the loss (-ELBO)
            train_op = opt.minimize(elbo_loss)
            # start the session
            sess = tf.Session()
            # initialize the variables
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())

            # Run the training loop
            train_handle = sess.run(training_iterator.string_handle())
            heldout_handle = sess.run(heldout_iterator.string_handle())
            test_handle = sess.run(test_iterator.string_handle())
            # Run the epochs
            for epoch in range(FLAGS.num_epochs):
                _ = sess.run([train_op, accuracy_update_op],
                             feed_dict={handle: train_handle})

                if epoch % 100 == 0:
                    loss_value, accuracy_value = sess.run(
                        [elbo_loss, accuracy], feed_dict={handle: train_handle})
                    loss_value_validation, accuracy_value_validation = sess.run(
                        [elbo_loss, accuracy], feed_dict={handle: heldout_handle}
                    )
                    print("Epoch: {:>3d} Loss: [{:.3f}, {:.3f}] Accuracy: [{:.3f}, {:.3f}]".format(
                        epoch, loss_value, loss_value_validation, accuracy_value, accuracy_value_validation))

                # Check if final epoch, if so return the validation loss for the last epoch             
                if epoch == FLAGS.num_epochs - 1:
                    final_loss, final_accuracy = sess.run(
                        [elbo_loss, accuracy], feed_dict={handle: heldout_handle}
                    )
                    print("Final loss: [{:.3f}, {:.3f}] Final accuracy: [{:.3f}, {:.3f}]".format(
                        loss_value, loss_value_validation, accuracy_value, accuracy_value_validation))
        with tf.name_scope("evaluate"):
            # interpolate the predictive distributions and get the percentiles to represent
            # an empirical credible interval for the predictions
            probs = np.asarray([sess.run((labels_distribution.probs),
                                         feed_dict={handle: heldout_handle})
                                for _ in range(FLAGS.num_monte_carlo)])
            predictions = np.squeeze(predictions)  # fix the dimensions into a flat matrix
            credible_intervals = []  # will be a matrix with with lower- and upper bound as columns
            # loop over the columns and compute the empirical credible interval
            modes = []
            for i in range(predictions.shape[1]):
                lb = np.percentile(predictions[:, i], 2.5)
                ub = np.percentile(predictions[:, i], 97.5)
                mode = np.mean(predictions[:, i])
                credible_intervals.append([lb, ub])
                modes.append(mode)
    # check how often the true value is inside the credible interval
    with load('data_main.npz') as data:
        print(data)
        labels = data["labels"]
        features = data["features"]
        train_range = int(TRAIN_PERCENTAGE * len(features))
        validation_labels = labels[train_range:]
        inside = 0
        SSE = 0
        for i in range(validation_labels.shape[0]):
            label = validation_labels[i]
            if label >= credible_intervals[i][0] and label <= credible_intervals[i][1]:
                inside += 1
                SSE += (label - modes[i]) ** 2
    print("MSE", SSE / validation_labels.shape[0])
    print(inside / validation_labels.shape[0])

# HYPERPARAMETER OPTIMIZATION

# Define the hyperparametric space (some form of prior by specyfying range)
fspace = {
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1),
    'num_hidden_layers': hp.uniform('num_hidden_layers', 1, 7 + 1),
    'num_neurons_per_layer': hp.uniform('num_neurons_per_layer', 5, 200 + 1),
    'activation_function': hp.choice('activation_function', ["sigmoid", "relu"]),
    'num_principal_components': hp.uniform('num_principal_components', 20, 500 + 1)
}

# Wrapper around the objective function, assigns the flag values from the trials
def wrapper(params):
    FLAGS.learning_rate = params['learning_rate']
    FLAGS.num_hidden_layers = int(params['num_hidden_layers'])
    FLAGS.num_neurons_per_layer = int(params['num_neurons_per_layer'])
    FLAGS.activation_function = params['activation_function']
    FLAGS.num_principal_components = int(params['num_principal_components'])
    FLAGS.batch_size = 44  # kept constant
    FLAGS.num_epochs = 10000  # kept constant
    FLAGS.num_monte_carlo = 50  # default value 
    return main(FLAGS)

def caller(argv):
    trials = Trials()
    best = fmin(fn=wrapper, space=fspace, algo=tpe.suggest, max_evals=1000, trials=trials)
    print("Best:", best)
    print("Trials:")
    for trial in trials.trials:
        print(trial)


tf.app.run(main=caller)
# tf.app.run()

